In [ ]:
# Will Bring the Dataset in the form tag: {Tokenized Sentences}

In [1]:
#  DATA PREPROCESSING CELL  #
#===========================#

import pandas as pd
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')


pd.set_option('display.max_colwidth', -1)


# Path to Initial Dataset
path = './quotes.json'

# Import JSON as Pandas Dataframe
df = pd.read_json(path, orient='columns')
# df.head(5)

# Remove list for all useless Tags
remove_list = ['attributed-no-source']

# Function to Remove Redundant Columns and Tags 
def curate_data(data):    
    data = data.drop(['Author', 'Popularity', 'Category'], axis = 1)
    data.drop_duplicates(subset ="Quote", keep = 'first', inplace = True) 
    data['Tags'] = data['Tags'].apply(lambda tags_list: [x for x in tags_list if x not in remove_list and not x.startswith('misattributed')])        
    return data


# Function to Apply Text Preprocessing
def preprocess(sentence):
    return [_ for _ in word_tokenize(sentence.lower().translate(str.maketrans('', '', string.punctuation))) if _ not in stop]




In [2]:
# Remove Redundant Data
curated_data = curate_data(df)
curated_data.index = range(len(curated_data))

# Write Curated Data to JSON
curated_data.to_json(r'./curated-data.json')

# Test with the first 10 Quotes to Implement Preprocessing
testdf = curated_data[:10]
testdf.Quote = testdf.Quote.apply(preprocess)
testdf = testdf.explode('Tags').reset_index(drop=True)
testdf.head(50)


# testdf['Quote'].apply(lambda x: x.lower()).apply(lambda x: nltk.word_tokenize(x)).head()

/home/kyriakos/anaconda3/envs/kwd_ext/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Quote,Tags
0,"[dont, cry, smile, happened]",cry
1,"[dont, cry, smile, happened]",crying
2,"[dont, cry, smile, happened]",experience
3,"[dont, cry, smile, happened]",happiness
4,"[dont, cry, smile, happened]",joy
5,"[dont, cry, smile, happened]",life
6,"[dont, cry, smile, happened]",optimism
7,"[dont, cry, smile, happened]",sadness
8,"[dont, cry, smile, happened]",smile
9,"[dont, cry, smile, happened]",smiling


In [26]:
from functools import reduce
# Identify Unique Tags
# Create flat list of Tags
tags = testdf['Tags'].tolist()
flat_tags = list(dict.fromkeys(tags))

a = set([1,2,3,3,3])
b = set([1,2,2,2])
list(a.union(b))

# assert all([type(x) == list for x in testdf.Quote])

# def create_dict(data):
#     return {tag: reduce(lambda x, y: list(set(x).union(set(y))), data.Quote[data.Tags == tag]) for tag in flat_tags}

def create_dict(data):
    return {tag: list(set(reduce(lambda x, y: x + y, data.Quote[data.Tags == tag]))) for tag in flat_tags}


a = create_dict(testdf)

print(a)
[type(_) for _ in a.values()]
assert all([type(x) == list for x in a.values()])

for i, d in enumerate(a.items()):
    if len(d[1]) != len(set(d[1])):
        print(i, d[0], sorted(d[1]), len(d[1]), "G", len(set(d[1])), set(d[1]))
assert all([len(x) == len(set([_ for _ in x])) for x in a.values()])


{'cry': ['smile', 'dont', 'happened', 'cry'], 'crying': ['smile', 'dont', 'happened', 'cry'], 'experience': ['smile', 'dont', 'happened', 'cry'], 'happiness': ['smile', 'dont', 'happened', 'cry'], 'joy': ['smile', 'dont', 'happened', 'cry'], 'life': ['hurtsing', 'make', 'watchinglove', 'times', 'deserve', 'got', 'dont', 'earth', 'heaven', 'selfish', 'happened', 'control', 'worst', 'youve', 'im', 'ta', 'youll', 'sure', 'smile', 'live', 'hell', 'cry', 'mistakes', 'theres', 'little', 'insecure', 'nobody', 'handle', 'dance', 'listeningand', 'hard', 'best', 'like', 'cant', 'impatient', 'never'], 'optimism': ['smile', 'dont', 'happened', 'cry'], 'sadness': ['smile', 'dont', 'happened', 'cry'], 'smile': ['smile', 'dont', 'happened', 'cry'], 'smiling ': ['smile', 'dont', 'happened', 'cry'], 'best': ['worst', 'mistakes', 'im', 'make', 'handle', 'times', 'little', 'insecure', 'dont', 'hard', 'deserve', 'sure', 'best', 'selfish', 'cant', 'hell', 'control', 'impatient'], 'love': ['youre', 'hurtsin

In [ ]:
# Create New Dataframe with Unique Tags as Columns
testdf3 = testdf.sort_values(by=['Tags'])
testdf4 = pd.DataFrame(columns=[flat_tags])



cond = testdf3.Tags == 'love'
rows = testdf3.loc[cond, :]
testdf4['love'] = testdf3
testdf4.head()

In [ ]:
import shorttext
#  TRAIN DATA LOADING CELL  #
#===========================#

# Subject Dataset
trainclassdict = shorttext.data.subjectkeywords()
print(trainclassdict)


# NIH RePORT Dataset
#trainclassdict = shorttext.data.nihreports()
#print(trainclassdict)

# Inaugural Addresses Dataset
#trainclassdict = shorttext.data.inaugural()


# User Defined Dataset (JSON)
#trainclassdict = shorttext.data.retrieve_jsondata_as_dict('./quotes.json')
#print(trainclassdict[0])
# User Defined Dataset (CSV Heading & at least 2 columns)
# trainclassdict = shorttext.data.retrieve_csvdata_as_dict('/path/to/file.csv')

